In [2]:
import nomic
import json
nomic.login("")
#- Scientific programming
import numpy as np
import pandas as pd
from tqdm import tqdm

#- Plotting
import matplotlib.pyplot as plt
import seaborn as sns

#- Data
from nomic import AtlasDataset

#- Processing
from fuzzywuzzy import fuzz
from unidecode import unidecode
import re
import datetime

dataset = AtlasDataset('hivemind/tweets-from-members-of-us-congress-from-all-time-updated-2024-08-14')

df = dataset.maps[0].data.df

#tags_df = dataset.maps[0].tags.df

#df = df.merge(tags_df, on='tweetId')
df.head()

/cis/home/tchen94/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
2024-08-27 11:37:54.494 | INFO     | nomic.dataset:__init__:768 - Loading existing dataset `hivemind/tweets-from-members-of-us-congress-from-all-time-updated-2024-08-14`.
2024-08-27 11:38:06.121 | WARNING  | nomic.data_operations:df:923 - Converting to pandas dataframe. This may materialize a large amount of data into memory.
2024-08-27 11:38:06.124 | INFO     | nomic.data_operations:_download_data:902 - Downloading data
100%|██████████| 129/129 [00:00<00:00, 793.40it/s]
2024-08-27 11:38:14.658 | INFO     | nomic.data_operations:_load_data:872 - Loading data
100%|██████████| 129/129 [00:02<00:00, 53.83it/s]


,tweetId,postedAt,name,state,party,twitter_lower,chamber,years,source,id,text
0,1065633444502224896,2018-11-22 15:49:46.338,Ken Buck R-CO,CO,R,repkenbuck,House,2017_2018,PolitWoops,1203633,Happy #Thanksgiving! I hope everyone has a cha...
1,1479108462504136704,2022-01-06 15:11:52.624,Kat Cammack R-FL,FL,R,repkatcammack,House,2021_2022,PolitWoops,2857207,"RT @MorningsMaria: .@RepKatCammack: ""The Democ..."
2,85108215381102592,2011-06-26 22:12:34.438,Dennis Ross R-FL,FL,R,repdennisross,House,2011_2012,PolitWoops,35353,@CEP_Observer will have to check that out.
3,925691081026621440,2017-11-01 11:48:46.882,Richard Hudson R-NC,NC,R,reprichhudson,House,2017_2018,PolitWoops,954870,RT @BGClubCab: Thanks to @CharlotteWBB for hel...
4,1435963943533568000,2021-09-09 13:50:57.715,Chip Roy R-TX,TX,R,repchiproy,House,2021_2022,PolitWoops,2324693,RT @chiproytx: No. https://t.co/STZW0DdDz1


In [2]:
df.head()

,tweetId,postedAt,name,state,party,twitter_lower,chamber,years,source,id,text
0,1065633444502224896,2018-11-22 15:49:46.338,Ken Buck R-CO,CO,R,repkenbuck,House,2017_2018,PolitWoops,1203633,Happy #Thanksgiving! I hope everyone has a cha...
1,1479108462504136704,2022-01-06 15:11:52.624,Kat Cammack R-FL,FL,R,repkatcammack,House,2021_2022,PolitWoops,2857207,"RT @MorningsMaria: .@RepKatCammack: ""The Democ..."
2,85108215381102592,2011-06-26 22:12:34.438,Dennis Ross R-FL,FL,R,repdennisross,House,2011_2012,PolitWoops,35353,@CEP_Observer will have to check that out.
3,925691081026621440,2017-11-01 11:48:46.882,Richard Hudson R-NC,NC,R,reprichhudson,House,2017_2018,PolitWoops,954870,RT @BGClubCab: Thanks to @CharlotteWBB for hel...
4,1435963943533568000,2021-09-09 13:50:57.715,Chip Roy R-TX,TX,R,repchiproy,House,2021_2022,PolitWoops,2324693,RT @chiproytx: No. https://t.co/STZW0DdDz1


In [3]:
embds = dataset.maps[0].embeddings.latent

2024-08-27 11:38:46.516 | INFO     | nomic.data_operations:_download_latent:550 - Downloading latent embeddings
100%|██████████| 129/129 [00:00<00:00, 657.44it/s]
2024-08-27 11:38:54.510 | INFO     | nomic.data_operations:latent:527 - Loading latent embeddings
100%|██████████| 129/129 [00:05<00:00, 23.94it/s]


In [4]:
embds=embds[:,[0,1]]

In [5]:
embds.shape

(3180890, 2)

In [5]:
df_2020 = df[df['postedAt'].dt.year == 2020]
row_index = df_2020.index
row_index


Int64Index([     14,      27,      31,      34,      36,      48,      51,
                 61,      65,      67,
            ...
            3180836, 3180844, 3180848, 3180853, 3180856, 3180859, 3180861,
            3180863, 3180866, 3180874],
           dtype='int64', length=412546)

In [13]:
embds_2020 = embds[row_index]
embds_2020.shape

(412546, 2)

In [10]:
df_2020['month']=df_2020['postedAt'].dt.month

/var/folders/rl/pr6f4j2d1_5913_lglcbwszc0000gn/T/ipykernel_6182/1828011652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['month']=df_2020['postedAt'].dt.month


In [16]:
df_2020 = df_2020.reset_index(drop=True)
embds_2020_df = embds_2020_df.reset_index(drop=True)
df_2020 = pd.concat([df_2020, embds_2020_df], axis=1)

In [15]:
import pandas as pd
# Convert embds_2020 ndarray to DataFrame
embds_2020_df = pd.DataFrame(embds_2020, columns=['embds_2020_col1', 'embds_2020_col2'])

df_2020 = pd.concat([df_2020, embds_2020_df], axis=1)

